# Декораторы

In [26]:
def foo(x):
    print(x + 1)


def decorator(foo):
    def _dec(*args, **kwargs):
        print("inside decorator")
        res = foo(*args, **kwargs)
        return res
    
    return _dec

In [27]:
foo(1)

2


In [28]:
foo = decorator(foo)
foo(1)

inside decorator
2


### В Python есть специальный синтаксис, который позволяет переместить модификацию функции ближе к моменту объявления функции:

In [41]:
@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [42]:
foo(1)

inside decorator
2


### Есть небольшая проблема:

In [44]:
foo.__name__, foo.__doc__

('_dec', None)

#### Еще "пропадает" ```__module__```
#### Что делать?

In [51]:
def decorator(foo):
    def _dec(*args, **kwargs):
        res = foo(*args, **kwargs)
        return res
    
    _dec.__name__ = foo.__name__
    _dec.__doc__ = foo.__doc__
    _dec.__module__ = foo.__module__
    
    return _dec


@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [52]:
foo.__name__, foo.__doc__

('foo', '\n    foo description\n    ')

#### Но это сложно, можно проще.
#### Воспользуемся декоратором ```wraps``` из пакета ```functools``` <i>(подробнее про этот пакет чуть позже)</i>

In [55]:
from functools import wraps


def decorator(foo):
    @wraps(foo)
    def _dec(*args, **kwargs):
        res = foo(*args, **kwargs)
        return res
    
    return _dec


@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [56]:
foo.__name__, foo.__doc__

('foo', '\n    foo description\n    ')

### Декораторов может быть несколько:

In [60]:
def dec1(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside dec1")
        res = func(*args, **kwargs)
        return res
        
    return _dec


def dec2(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside dec2")
        res = func(*args, **kwargs)
        return res
        
    return _dec

In [7]:
@dec2
@dec1
def func(arg1, arg2):
    pass

In [9]:
func(1, 2)

inside dec2
inside dec1


#### Это же самое, что:

In [12]:
def func(arg1, arg2):
    pass

In [13]:
func = dec2(dec1(func))

In [14]:
func(1, 2)

inside dec2
inside dec1


### Вопрос [?]

In [75]:
flag = False


def decorator(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside _dec")
        res = func(*args, **kwargs)
        return res
    
    return func if flag else _dec


@decorator
def func():
    pass

In [76]:
func()

inside _dec


#### Что будет?

In [ ]:
flag = True
func()

### В декоратор можно передавать аргументы:

In [80]:
def decorator_with_args(dec_argument):
    def _decorator(func):
        @wraps(func)
        def _dec(*args, **kwargs):
            print(f"inside decorator; {dec_argument}")
            res = func(*args, **kwargs)
            return res
        
        return _dec
    return _decorator

In [81]:
@decorator_with_args("hop hey lala ley")
def func(x):
    pass

In [82]:
func(None)

inside decorator; hop hey lala ley


#### Это то же самое, что:

In [88]:
def decorator_with_args(dec_argument):
    def _decorator(func):
        @wraps(func)
        def _dec(*args, **kwargs):
            print(f"inside decorator; {dec_argument}")
            res = func(*args, **kwargs)
            return res
        
        return _dec
    return _decorator

In [89]:
def func(x):
    pass

In [90]:
decorator = decorator_with_args("hop hey lala ley")  # получим тут декоратор
func = decorator(func)  # получим модифицированную функцию

In [91]:
func(None)

inside decorator; hop hey lala ley


#### А если мы хотим опциональные аргументы?

In [61]:
def decorator_with_optional_arguments(func=None, *, dec_argument="default"):
    if func is None:
        return lambda func: decorator_with_optional_arguments(func, dec_argument=dec_argument)
    @wraps(func)
    def _dec(*args, **kwargs):
        print(f"inside decorator; {dec_argument}")
        res = func(*args, **kwargs)
        return res
    
    return _dec

In [36]:
@decorator_with_optional_arguments(dec_argument="Life is beatiful")
def func1():
    pass


func1()

inside decorator; Life is beatiful


In [37]:
@decorator_with_optional_arguments
def func2():
    pass


func2()

inside decorator; default


### Примеры полезных декораторов

In [ ]:
# TODO

# Области видимости

In [ ]:
# TODO

# @classmethod и @staticmethod

In [98]:
class A(object):
    count_ = 100500
    
    def foo(self, x):
        print(f"foo with self={self} and x={x}")

    @classmethod
    def class_foo(cls, x):
        print(f"class foo with cls={cls} and local_variables={dir()} \
including count_={cls.count_}")

    @staticmethod
    def static_foo(x):
        print(f"static foo x={x} and local_variables={dir()}")  


a=A()

In [99]:
a.foo("some_x")

foo with self=<__main__.A object at 0x7fdc8460cc88> and x=some_x


In [100]:
a.class_foo("some_x")

class foo with cls=<class '__main__.A'> and local_variables=['cls', 'x'] including count_=100500


In [101]:
A.class_foo("some_x")

class foo with cls=<class '__main__.A'> and local_variables=['cls', 'x'] including count_=100500


In [102]:
a.static_foo("some_x")

static foo x=some_x and local_variables=['x']


In [103]:
A.static_foo("some_x")

static foo x=some_x and local_variables=['x']


# Дескрипторы

In [ ]:
# TODO

# Наследование и MRO

In [ ]:
# TODO

# Декораторы классов

In [ ]:
# TODO